# Capstone Project - Recommending Toronto Neighbourhoods to Immigrants

### Introduction / Business Problem : 
There is a large scale migration from India to Canada and Toronto is one of the major destination of these new migrants.The new migrants have to first decide, which Neighbourhood they should choose to stay based on their preferences of venues.

### Objective : 
The project should help the new immigrants to Toronto, Canada decide on the Neighbourhood that they should choose to stay. 
This project can help the immigrant with the following outcomes:

### 1. Recommendation of Neighbourhoods basis an immigrants preference of venues


### Provide overall information to the immigrant on the Neighbourhoods including,

### 2. Top 10 venues per Neighbourhood
### 3. Clustering of similar neighbourhoods basis the venues

### Data Sources :

Toronto Neighbourhoods Post Codes https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695

Co-Ordinates Data ( Geospatial_Coordinates.csv )

Four Square Data on Toronto Neighbourhoods/ Venues 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Neighbourhood_latitude, 
    Neighbourhood_longitude, 
    radius, 
    LIMIT)

## 1. Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


## 2. Extracting data from wikipedia page and updating in a Pandas DataFrame

In [2]:
url1 = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695'

In [3]:
soup = BeautifulSoup(requests.get (url1).text)

In [4]:
pretty_soup = soup.prettify ()

In [5]:
df_1 = pd.read_html (pretty_soup, header = 0)

In [6]:
Neighbourhoods = df_1[0]

## 3. Data Cleaning and Organising

In [7]:
Neighbourhoods.shape

(288, 3)

In [8]:
Neighbourhoods.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postcode         288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [9]:
Neighbourhoods.head (3)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


In [10]:
P = Neighbourhoods.Postcode.nunique ()
B = Neighbourhoods.Borough.nunique ()
N = Neighbourhoods.Neighbourhood.nunique ()

print (' Postcode      :', P, '\n','Borough       :', B, '\n','Neighbourhood :', N )

 Postcode      : 180 
 Borough       : 12 
 Neighbourhood : 209


Removing 'Not assigned' from 'Borough'

In [11]:
Neighbourhoods [Neighbourhoods.Borough == 'Not assigned'].count ()

Postcode         77
Borough          77
Neighbourhood    77
dtype: int64

In [12]:
Neighbourhoods.Borough = Neighbourhoods.Borough.replace ('Not assigned', np.nan)

In [13]:
Neighbourhoods.head ()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
Neighbourhoods.dropna (inplace = True)

In [15]:
Neighbourhoods.head ()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [16]:
Neighbourhoods.shape

(211, 3)

Substituting 'Not assigned' in 'Neighbourhood'

In [17]:
np.sum (Neighbourhoods.Neighbourhood == 'Not assigned')

1

In [18]:
Neighbourhoods[Neighbourhoods.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [19]:
Neighbourhoods [Neighbourhoods.Neighbourhood == 'Not assigned'] = "Queen's Park"

In [20]:
np.sum (Neighbourhoods.Neighbourhood == 'Not assigned')

0

In [21]:
Neighbourhoods.head ()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [22]:
Neighbourhoods.shape

(211, 3)

Combine rows that have same PostalCode and Borough

In [23]:
Grouped = Neighbourhoods.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)

In [24]:
Grouped=Grouped.sample(frac=1).reset_index()

In [25]:
Grouped['Neighbourhood']=Grouped['Neighbourhood'].str.join(', ')

In [26]:
Grouped.head ()

,Postcode,Borough,Neighbourhood
0,M6N,York,"The Junction North, Runnymede"
1,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village"
2,M9A,Etobicoke,Islington Avenue
3,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
4,M4S,Central Toronto,Davisville


In [27]:
Grouped.shape

(103, 3)

Adding location coordinates

In [28]:
coord = pd.read_csv ('Geospatial_Coordinates.csv')

In [29]:
coord.head ()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
Merged = pd.merge (Neighbourhoods, coord, left_on = 'Postcode', right_on = 'Postal Code')

In [31]:
Merged.head ()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763


In [32]:
Merged.drop ('Postal Code', inplace = True, axis = 1)

Dataframe with location coordinates

In [33]:
Merged.head () 

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [34]:
Merged.isnull ().sum ().sum ()

0

In [35]:
P2 = Merged.Postcode.nunique ()
B2 = Merged.Borough.nunique ()
N2 = Merged.Neighbourhood.nunique ()

print (' Postcode      :', P2, '\n','Borough       :', B2, '\n','Neighbourhood :', N2 )
print ('Shape of dataframe: ',Merged.shape)

 Postcode      : 102 
 Borough       : 10 
 Neighbourhood : 208
Shape of dataframe:  (210, 5)


### 3.1 Neighbourhoods on Toronto Map Visualization

In [36]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [37]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Merged['Latitude'], Merged['Longitude'], Merged['Borough'], Merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2 Neighbourhoods DataFrame Preparation

In [38]:
CLIENT_ID = '0QAXTTYPA0GE3XVMOMWJA30KKWQFONE4UKMMMKTSX4UONGWM' # your Foursquare ID
CLIENT_SECRET = 'RCSW0XRSYLZQSBOZGJZTEUJABYVGZPZV5G3JR2C5CNOUJVZI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0QAXTTYPA0GE3XVMOMWJA30KKWQFONE4UKMMMKTSX4UONGWM
CLIENT_SECRET:RCSW0XRSYLZQSBOZGJZTEUJABYVGZPZV5G3JR2C5CNOUJVZI


Exploring one Neighbourhood in database

In [39]:
Merged.loc[0, 'Neighbourhood']

'Parkwoods'

In [40]:
Neighbourhood_latitude = Merged.loc [0,'Latitude']

In [41]:
Neighbourhood_longitude = Merged.loc [0,'Longitude']

In [42]:
Neighbourhood_name = Merged.loc[0, 'Neighbourhood']

In [43]:
print ('Latitude and Longitude is {} and {} for {}'.format (Neighbourhood_latitude,Neighbourhood_longitude,Neighbourhood_name))

Latitude and Longitude is 43.7532586 and -79.3296565 for Parkwoods


In [44]:
# type your answer here
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Neighbourhood_latitude, 
    Neighbourhood_longitude, 
    radius, 
    LIMIT)

In [45]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0QAXTTYPA0GE3XVMOMWJA30KKWQFONE4UKMMMKTSX4UONGWM&client_secret=RCSW0XRSYLZQSBOZGJZTEUJABYVGZPZV5G3JR2C5CNOUJVZI&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [46]:
results = requests.get (url).json ()

In [47]:
venues = results['response']['groups'][0]['items']

In [48]:
nearby_venues = json_normalize(venues) # flatten JSON

In [49]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

In [52]:
nearby_venues.columns

Index(['venue.name', 'venue.categories', 'venue.location.lat',
       'venue.location.lng'],
      dtype='object')

In [53]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [54]:
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,649 Variety,Convenience Store,43.754513,-79.331942
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [55]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


### 3.3 Extracting nearby venues for Neighbourhoods from Four Square

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    print ('done')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
toronto_venues = getNearbyVenues(names=Merged['Neighbourhood'],latitudes=Merged['Latitude'],longitudes=Merged['Longitude'])

done


In [58]:
toronto_venues.shape

(4237, 7)

In [59]:
toronto_venues.head ()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


## 4. Neighbourhoods Data Analysis

In [60]:
toronto_onehot = pd.get_dummies (toronto_venues [['Venue Category']], prefix = "", prefix_sep = "")

In [61]:
toronto_onehot ['Neighbourhood'] = toronto_venues ['Neighbourhood']

In [62]:
toronto_onehot.columns

Index(['Accessories Store', 'Afghan Restaurant', 'Airport',
       'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       'Airport Service', 'Airport Terminal', 'American Restaurant',
       'Antique Shop',
       ...
       'Vegetarian / Vegan Restaurant', 'Video Game Store', 'Video Store',
       'Vietnamese Restaurant', 'Warehouse Store', 'Wine Bar', 'Wings Joint',
       'Women's Store', 'Yoga Studio', 'Neighbourhood'],
      dtype='object', length=265)

In [63]:
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

In [64]:
toronto_onehot.head ()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [65]:
toronto_onehot.shape

(4237, 265)

In [66]:
toronto_grouped = round (toronto_onehot.groupby ('Neighbourhood').mean ().reset_index (),4)

In [69]:
toronto_grouped.head ()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0213,0.0,0.0,0.0106,0.0106,0.0,0.0106,0.0,0.0,0.0,0.0,0.0,0.0,0.0213,0.0,0.0106,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0213,0.0,0.0106,0.0106,0.0,0.0,0.0,0.0106,0.0106,0.0106,0.0,

In [70]:
toronto_grouped.shape

(204, 265)

In [71]:
toronto_grouped_transpose = toronto_grouped.transpose ()

In [72]:
toronto_grouped_transpose.head ()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203
Neighbourhood,Adelaide,Agincourt,Agincourt North,Albion Gardens,Alderwood,Bathurst Manor,Bathurst Quay,Bayview Village,Beaumond Heights,Bedford Park,Berczy Park,Birch Cliff,Bloordale Gardens,Brockton,Business Reply Mail Processing Centre 969 Eastern,CFB Toronto,CN Tower,Cabbagetown,Caledonia-Fairbanks,Canada Post Gateway Processing Centre,Cedarbrae,Central Bay Street,Chinatown,Christie,Church and Wellesley,Clairlea,Clarks Corners,Cliffcrest,Cliffside,Cliffside West,Cloverdale,Commerce Court,Davisville,Davisville North,Deer Park,Del Ray,Design Exchange,Don Mills North,Don Mills South,Dorset Park,Dovercourt Village,Downsview,Downsview Central,Downsview East,Downsview North,Downsview Northwest,Downsview West,Dufferin,East Birchmount Park,East Toronto,Emery,Eringate,Exhibition Place,Fairview,First Canadian Place,Flemingdon Park,Forest Hill North,Forest Hill SE,Forest Hill West,Garden District,Glencairn,Golden Mile,Grange Park,Guildwood,Harbord,Harbourfront,Harbourfront East,Harbourfront West,Henry Farm,High Park,Highland Creek,Hillcrest Village,Humber Bay,Humber Bay Shores,Humber Summit,Humbergate,Humberlea,Humewood-Cedarvale,India Bazaar,Ionview,Island airport,Islington,Jamestown,Keelesdale,Kennedy Park,Kensington Market,King,King and Spadina,King's Mill Park,Kingsview Village,Kingsway Park South East,Kingsway Park South West,L'Amoreaux East,L'Amoreaux West,Lawrence Heights,Lawrence Manor,Lawrence Manor East,Lawrence Park,Leaside,Little Portugal,Long Branch,Malvern,Markland Wood,Martin Grove,Martin Grove Gardens,Maryvale,Milliken,Mimico NE,Mimico NW,Mimico South,Montgomery Road,Moore Park,Morningside,Mount Dennis,Mount Olive,New Toronto,North Midtown,North Park,North Toronto West,Northwest,Northwood Park,Oakridge,Old Burnhamthorpe,Old Mill North,Old Mill South,Oriole,Parkdale,Parkdale Village,Parkview Hill,Parkwoods,Port Union,Princess Gardens,Railway Lands,Rathnelly,Regent Park,Richmond,Richview Gardens,Riverdale,Roncesvalles,Rosedale,Roselawn,Rouge,Rouge Hill,Royal York South East,Royal York South West,Runnymede,Ryerson,Scarborough Town Centre,Scarborough Village,Scarborough Village West,Silver Hills,Silverstone,Silverthorn,South Hill,South Niagara,South Steeles,South of Bloor,St. James Town,St. Phillips,Steeles East,Stn A PO Boxes 25 The Esplanade,Studio District,Sullivan,Summerhill East,Summerhill West,Sunnylea,Swansea,Tam O'Shanter,The Annex,The Beaches,The Beaches West,The Danforth West,The Junction North,The Junction South,The Kingsway,The Queensway East,The Queensway West,Thistletown,Thorncliffe Park,Toronto Dominion Centre,Toronto Islands,Trinity,Underground city,Union Station,University of Toronto,Upwood Park,Victoria Hotel,Victoria Village,West Deane Park,West Hill,Westmount,Weston,Wexford,Wexford Heights,Willowdale South,Willowdale West,Wilson Heights,Woburn,Woodbine Gardens,Woodbine Heights,York Mills,York Mills West,York University,Yorkville
Accessories Store,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0833,0.0833,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [73]:
for i in toronto_grouped_transpose.columns:
    V = toronto_grouped_transpose.iloc [1:,i].sort_values (ascending = False).head ()
    # print ('Neighbourhood :', toronto_grouped_transpose.iloc [0,i],'\n','Venue & Frequency','\n\n', V)
    # print ('\n')

In [74]:
toronto_dict = {}

# 5.1 Top 10 venues per Neighbourhood - Project Outcome. 2

In [75]:
for i in range(len(toronto_grouped_transpose.columns)):
    value = toronto_grouped_transpose.iloc [1:,i].sort_values (ascending = False).head (10).index.to_list ()
    key = toronto_grouped_transpose.iloc [0,i]
    toronto_dict.update ({key:value})

### Updating top venues per Neighbourhood to a dataframe

In [77]:
toronto_df = pd.DataFrame (toronto_dict).transpose ()

In [78]:
toronto_df.index.name = 'Neighbourhood'
toronto_df.columns = ['First', 'Second', 'Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eigth', 'Nineth', 'Tenth']

In [79]:
toronto_df.head (3)

,First,Second,Third,Fourth,Fifth,Sixth,Seventh,Eigth,Nineth,Tenth
Neighbourhood,,,,,,,,,,
Adelaide,Coffee Shop,Café,Restaurant,Deli / Bodega,Hotel,Thai Restaurant,Clothing Store,Gym,Bookstore,Concert Hall
Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
Agincourt North,Park,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


# 5.2 Clustering of similar neighbourhoods basis venues - Project Outcome. 3

In [80]:
# set number of clusters 
kclusters = 5

In [81]:
toronto_grouped_clustering = toronto_grouped.drop ('Neighbourhood',1)

In [82]:
# run k-means clustering
kmeans = KMeans (n_clusters = kclusters, random_state = 0)

In [83]:
kmeans.fit (toronto_grouped_clustering)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [84]:
kmeans.labels_ [0:10]

array([3, 3, 1, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [85]:
toronto_df.insert (0, 'Cluster Labels', kmeans.labels_)

In [86]:
toronto_df.tail (3)

,Cluster Labels,First,Second,Third,Fourth,Fifth,Sixth,Seventh,Eigth,Nineth,Tenth
Neighbourhood,,,,,,,,,,,
York Mills West,1,Park,Bank,Convenience Store,Bar,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
York University,3,Falafel Restaurant,Miscellaneous Shop,Coffee Shop,Massage Studio,Caribbean Restaurant,Bar,Dog Run,Diner,Discount Store,Distribution Center
Yorkville,3,Café,Sandwich Place,Coffee Shop,Liquor Store,Cosmetics Shop,Pub,BBQ Joint,Burger Joint,History Museum,Middle Eastern Restaurant


In [87]:
toronto_merged = Merged.join (toronto_df, on = 'Neighbourhood')

In [88]:
toronto_merged.tail (3)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,First,Second,Third,Fourth,Fifth,Sixth,Seventh,Eigth,Nineth,Tenth
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999,3.0,Social Club,Burger Joint,Wings Joint,Grocery Store,Gym,Bakery,Fast Food Restaurant,Hardware Store,Sandwich Place,Convenience Store
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999,3.0,Social Club,Burger Joint,Wings Joint,Grocery Store,Gym,Bakery,Fast Food Restaurant,Hardware Store,Sandwich Place,Convenience Store
209,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999,3.0,Social Club,Burger Joint,Wings Joint,Grocery Store,Gym,Bakery,Fast Food Restaurant,Hardware Store,Sandwich Place,Convenience Store


In [89]:
toronto_merge = toronto_merged.dropna ()

In [90]:
toronto_merge.isnull ().sum ().sum ()

0

In [91]:
toronto_merge.info ()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 209
Data columns (total 16 columns):
Postcode          206 non-null object
Borough           206 non-null object
Neighbourhood     206 non-null object
Latitude          206 non-null float64
Longitude         206 non-null float64
Cluster Labels    206 non-null float64
First             206 non-null object
Second            206 non-null object
Third             206 non-null object
Fourth            206 non-null object
Fifth             206 non-null object
Sixth             206 non-null object
Seventh           206 non-null object
Eigth             206 non-null object
Nineth            206 non-null object
Tenth             206 non-null object
dtypes: float64(3), object(13)
memory usage: 27.4+ KB


In [92]:
toronto_merge.tail (3)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,First,Second,Third,Fourth,Fifth,Sixth,Seventh,Eigth,Nineth,Tenth
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999,3.0,Social Club,Burger Joint,Wings Joint,Grocery Store,Gym,Bakery,Fast Food Restaurant,Hardware Store,Sandwich Place,Convenience Store
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999,3.0,Social Club,Burger Joint,Wings Joint,Grocery Store,Gym,Bakery,Fast Food Restaurant,Hardware Store,Sandwich Place,Convenience Store
209,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999,3.0,Social Club,Burger Joint,Wings Joint,Grocery Store,Gym,Bakery,Fast Food Restaurant,Hardware Store,Sandwich Place,Convenience Store


### Clusters Visualization

In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [94]:
# add markers to the map
markers_colors = []

In [95]:
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighbourhood'], toronto_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow [int(cluster-1)],
        fill=True,
        fill_color=rainbow [int (cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)

In [96]:
map_clusters

### Checking a Cluster post analysis

In [97]:
toronto_merge.loc [toronto_merge['Cluster Labels'] == 1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,First,Second,Third,Fourth,Fifth,Sixth,Seventh,Eigth,Nineth,Tenth
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Convenience Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Dog Run
27,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,1.0,Trail,Park,Field,Hockey Arena,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
37,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,1.0,Women's Store,Pool,Park,Colombian Restaurant,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
59,M4J,East York,East Toronto,43.685347,-79.338106,1.0,Park,Convenience Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Deli / Bodega,Doner Restaurant
69,M3K,North York,CFB Toronto,43.737473,-79.464763,1.0,Park,Airport,Playground,Snack Place,Colombian Restaurant,Comfort Food Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
70,M3K,North York,Downsview East,43.737473,-79.464763,1.0,Park,Airport,Playground,Snack Place,Colombian Restaurant,Comfort Food Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore
88,M6L,North York,Downsview,43.713756,-79.490074,1.0,Park,Bakery,Construction & Landscaping,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
89,M6L,North York,North Park,43.713756,-79.490074,1.0,Park,Bakery,Construction & Landscaping,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
90,M6L,North York,Upwood Park,43.713756,-79.490074,1.0,Park,Bakery,Construction & Landscaping,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
105,M9M,North York,Emery,43.724766,-79.532242,1.0,Paper / Office Supplies Store,Baseball Field,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant


# 5.3 Recommendation of Neighbourhoods basis an immigrants preference of venues - Project Outcome. 1

In [98]:
venue_names = np.array (toronto_grouped_transpose.index)

In [99]:
venue_names_df = pd.DataFrame (index = venue_names [1:])

## Individual immigrant preference of venues in a Neighbourhood
### 5 Perferences of venues with rating out of 5 per venue to be taken from new immigrant

In [100]:
immigrant = {'venues' : ['Airport','Fruit & Vegetable Store','Medical Center', 'Lake', 'Wine Shop'], 'rating': [3,4,4,3,3]}

In [101]:
immigrant_df = pd.DataFrame (immigrant)

In [102]:
immigrant_df = immigrant_df.set_index ('venues')

In [103]:
immigrant_df.head ()

,rating
venues,
Airport,3
Fruit & Vegetable Store,4
Medical Center,4
Lake,3
Wine Shop,3


In [104]:
venue_names_df.head ()

""
Accessories Store
Afghan Restaurant
Airport
Airport Food Court
Airport Gate


In [105]:
immigrant_venue_names = venue_names_df.join (immigrant_df)

In [106]:
immigrant_venue_names.head ()

,rating
Accessories Store,NaN
Afghan Restaurant,NaN
Airport,3.0
Airport Food Court,NaN
Airport Gate,NaN


In [107]:
immigrant_venue_names = immigrant_venue_names.fillna (0)

In [108]:
immigrant_venue_names.head ()

,rating
Accessories Store,0.0
Afghan Restaurant,0.0
Airport,3.0
Airport Food Court,0.0
Airport Gate,0.0


In [109]:
immigrant_venue_names.shape

(264, 1)

In [110]:
toronto_group = toronto_grouped.copy ()

In [111]:
toronto_group = toronto_group.set_index ('Neighbourhood')

In [112]:
toronto_group.shape

(204, 264)

In [113]:
P = np.dot (toronto_group,immigrant_venue_names)

In [114]:
P.shape

(204, 1)

In [115]:
toronto_group ['score'] = P

In [116]:
toronto_group.head ()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,score
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [117]:
immigrant_preference = toronto_group ['score'].sort_values (ascending = False)

In [118]:
immigrant_preference.head (10)

Neighbourhood
CFB Toronto         0.7500
Downsview East      0.7500
Guildwood           0.5716
West Hill           0.5716
Morningside         0.5716
Railway Lands       0.1875
Bathurst Quay       0.1875
CN Tower            0.1875
Island airport      0.1875
King and Spadina    0.1875
Name: score, dtype: float64

## Above are recommended 10 Neighbourhoods to the immigrant. Higher score means more preference.